In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Import Neccessary Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from math import log, sqrt
from collections import defaultdict
from sklearn.linear_model import LinearRegression, Ridge, Lasso
%matplotlib inline

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
sales = sales.sort_values(['sqft_living','price'])

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

reg_all = Lasso(alpha=5e2,normalize=True)
model_all = reg_all.fit(sales[all_features], sales['price'])

Find what features had non-zero weight.

In [5]:
data = {'Selected Features' : all_features, 'Coefficients' : model_all.coef_}
selected_features = pd.DataFrame(data=data)
print(selected_features)
selected_features = selected_features[selected_features['Coefficients'] > 0]

print('\nFeatures with non-zero coefficients are : \n\n' ,selected_features)

   Selected Features  Coefficients
0           bedrooms      0.000000
1    bedrooms_square      0.000000
2          bathrooms      0.000000
3        sqft_living    134.439314
4   sqft_living_sqrt      0.000000
5           sqft_lot      0.000000
6      sqft_lot_sqrt      0.000000
7             floors      0.000000
8      floors_square      0.000000
9         waterfront      0.000000
10              view  24750.004586
11         condition      0.000000
12             grade  61749.103091
13        sqft_above      0.000000
14     sqft_basement      0.000000
15          yr_built     -0.000000
16      yr_renovated      0.000000

Features with non-zero coefficients are : 

    Selected Features  Coefficients
3        sqft_living    134.439314
10              view  24750.004586
12             grade  61749.103091


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection

<font color = 'steelblue'><b> Quiz :  Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights?

<font color = 'mediumvioletred'><b> Answer : `sqft_living`,	`view` and `grade` were features selected by LASSO  </b></font>

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [6]:
test = pd.read_csv('wk3_kc/wk3_kc_house_test_data.csv', dtype=dtype_dict)
train = pd.read_csv('wk3_kc/wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc/wk3_kc_house_valid_data.csv', dtype=dtype_dict)

Make sure to create the 4 features as we did earlier:

In [7]:
test['sqft_living_sqrt'] = test['sqft_living'].apply(sqrt)
test['sqft_lot_sqrt'] = test['sqft_lot'].apply(sqrt)
test['bedrooms_square'] = test['bedrooms']*test['bedrooms']
test['floors_square'] = test['floors']*test['floors']

train['sqft_living_sqrt'] = train['sqft_living'].apply(sqrt)
train['sqft_lot_sqrt'] = train['sqft_lot'].apply(sqrt)
train['bedrooms_square'] = train['bedrooms']*train['bedrooms']
train['floors_square'] = train['floors']*train['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

Now for each l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type np.logspace(1, 7, num=13).)

* Learn a model on TRAINING data using the specified l1_penalty. Make sure to specify normalize=True in the constructor
* Compute the RSS on VALIDATION for the current model (print or save the RSS)

Report which `l1_penalty` produced the lowest RSS on validation data.

In [8]:
L1 = []
RSS_list = []
for l1_penalty in np.logspace(1, 7, num=13) :
    reg_train = Lasso(alpha=l1_penalty, normalize=True)
    model_train = reg_train.fit(train[all_features], train['price'])
    predictions = reg_train.predict(validation[all_features])
    
    residuals = predictions - validation['price']
    RSS = np.sum(np.square(residuals))
    print('L1 Penalty : ', l1_penalty, '\tRSS : ', RSS, '\n')
    L1.append(l1_penalty)
    RSS_list.append(RSS)
    
    
'''The code below is used to answer the quiz question as below'''
# Create a dictionary with the keys as 'L1' and the values as 'RSS_List'
L1_list = dict(zip(L1,RSS_list))

# Get the key for the minimum value in the dictionary
best_L1 = min(L1_list, key=L1_list.get)

print('\nBest RSS on Test data is with ', best_L1, ' with RSS of ', min(RSS_list))


L1 Penalty :  10.0 	RSS :  398213327300135.0 

L1 Penalty :  31.622776601683793 	RSS :  399041900253346.8 

L1 Penalty :  100.0 	RSS :  429791604072559.56 

L1 Penalty :  316.22776601683796 	RSS :  463739831045121.06 

L1 Penalty :  1000.0 	RSS :  645898733633800.8 

L1 Penalty :  3162.2776601683795 	RSS :  1222506859427163.0 

L1 Penalty :  10000.0 	RSS :  1222506859427163.0 

L1 Penalty :  31622.776601683792 	RSS :  1222506859427163.0 

L1 Penalty :  100000.0 	RSS :  1222506859427163.0 

L1 Penalty :  316227.7660168379 	RSS :  1222506859427163.0 

L1 Penalty :  1000000.0 	RSS :  1222506859427163.0 

L1 Penalty :  3162277.6601683795 	RSS :  1222506859427163.0 

L1 Penalty :  10000000.0 	RSS :  1222506859427163.0 


Best RSS on Test data is with  10.0  with RSS of  398213327300135.0


<font color = 'steelblue'><b> Quiz :  Which was the best value for the l1_penalty, i.e. which value of l1_penalty produced the lowest RSS on VALIDATION data?

<font color = 'mediumvioletred'><b> Answer : The best value for `best_L1` is {{best_L1}} since it produced the lowest RSS on VALIDATION data </b></font>

Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty.

In [9]:
reg_test = Lasso(alpha=10, normalize=True,max_iter=10000)
model_test = reg_test.fit(test[all_features], test['price'])
predictions = reg_test.predict(test[all_features])
    
residuals = predictions - test['price']
RSS = np.sum(np.square(residuals))
print('RSS on TEST data : ', RSS)
'''The code below is used to answer the quiz question as below'''

if model_test.intercept_ != 0 : 
    total_count = np.count_nonzero(model_test.coef_) + 1
    print('Total number of non-zero coefficients : ', total_count)

elif model_test.intercept_ == 0 :
    total_count = np.count_nonzero(model_test.coef_)
    print('Total number of non-zero coefficients : ', total_count)

RSS on TEST data :  97246664646602.88
Total number of non-zero coefficients :  16


<font color = 'steelblue'><b> Quiz :  Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero

<font color = 'mediumvioletred'><b> Answer : The total number of non-zero coefficients is {{total_count}} </b></font>

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
* Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
* Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.
* Assign 7 to the variable ‘max_nonzeros’.


Exploring large range of l1_penalty

For l1_penalty in np.logspace(1, 4, num=20):

* Fit a regression model with a given l1_penalty on TRAIN data. Add "alpha=l1_penalty" and "normalize=True" to the parameter list.
* Extract the weights of the model and count the number of nonzeros. Take account of the intercept as we did earlier, adding 1 whenever the intercept is nonzero. Save the number of nonzeros to a list.

In [10]:
max_nonzeros = 7
not_zeros = 0
L1_penalties = []
RSS_ = []
non_zeros = []

for l1_penalty in np.logspace(1, 4, num=20):
    reg_limited = Lasso(alpha=l1_penalty,normalize=True)
    model_limited = reg_limited.fit(train[all_features], train['price'])
    
    predictions = reg_limited.predict(validation[all_features])
    
    residuals = predictions - validation['price']
    RSS = np.sum(np.square(residuals))
    
    if model_limited.intercept_ != 0 : 
        not_zeros = np.count_nonzero(model_limited.coef_) + 1
        print('L1 Penalty : ', l1_penalty, '\tRSS : ', RSS, '\nNumber of non-zero coefficients : ', 
              not_zeros, '\n')
    
    elif model_limited.intercept_ == 0 :
        not_zeros = np.count_nonzero(model_limited.coef_)
        print('L1 Penalty : ', l1_penalty, '\nNumber of non-zero coefficients : ', not_zeros, '\n')
    
    L1_penalties.append(l1_penalty)
    RSS_.append(RSS)
    non_zeros.append(not_zeros)

L1 Penalty :  10.0 	RSS :  398213327300135.0 
Number of non-zero coefficients :  15 

L1 Penalty :  14.38449888287663 	RSS :  396831833943813.5 
Number of non-zero coefficients :  15 

L1 Penalty :  20.6913808111479 	RSS :  396210901853184.3 
Number of non-zero coefficients :  15 

L1 Penalty :  29.76351441631318 	RSS :  398215534574786.06 
Number of non-zero coefficients :  15 

L1 Penalty :  42.81332398719393 	RSS :  406877258520204.6 
Number of non-zero coefficients :  13 

L1 Penalty :  61.58482110660264 	RSS :  424647490490609.44 
Number of non-zero coefficients :  12 

L1 Penalty :  88.58667904100822 	RSS :  427906308934484.8 
Number of non-zero coefficients :  11 

L1 Penalty :  127.42749857031335 	RSS :  435374677102680.6 
Number of non-zero coefficients :  10 

L1 Penalty :  183.29807108324357 	RSS :  443107216261395.56 
Number of non-zero coefficients :  7 

L1 Penalty :  263.6650898730358 	RSS :  454176669662635.25 
Number of non-zero coefficients :  6 

L1 Penalty :  379.26

In [11]:
# Create a dataframe with 'non_zeros', 'RSS_' and 'L1_penalties'
data = {'non_zeros' : non_zeros, 'l1' : L1_penalties, 'rss' :RSS_}
df = pd.DataFrame(data=data)

print('MAX DF Before Dropping : \n', df)
df_max = df[df['non_zeros'] < max_nonzeros]
print('\nMAX DF After Dropping :\n', df_max, '\n')

print('\nMIN DF Before Dropping : \n', df)
df_min = df[df['non_zeros'] > max_nonzeros]
print('\nMIN DF After Dropping :\n', df_min, '\n')

MAX DF Before Dropping : 
     non_zeros            l1           rss
0          15     10.000000  3.982133e+14
1          15     14.384499  3.968318e+14
2          15     20.691381  3.962109e+14
3          15     29.763514  3.982155e+14
4          13     42.813324  4.068773e+14
5          12     61.584821  4.246475e+14
6          11     88.586679  4.279063e+14
7          10    127.427499  4.353747e+14
8           7    183.298071  4.431072e+14
9           6    263.665090  4.541767e+14
10          6    379.269019  4.781330e+14
11          6    545.559478  5.313972e+14
12          5    784.759970  5.940433e+14
13          3   1128.837892  6.740592e+14
14          3   1623.776739  8.026094e+14
15          2   2335.721469  1.061255e+15
16          1   3359.818286  1.222507e+15
17          1   4832.930239  1.222507e+15
18          1   6951.927962  1.222507e+15
19          1  10000.000000  1.222507e+15

MAX DF After Dropping :
     non_zeros            l1           rss
9           6    263.66

Out of this large range, we want to find the two ends of our desired narrow range of l1_penalty. At one end, we will have l1_penalty values that have too few non-zeros, and at the other end, we will have an l1_penalty that has too many non-zeros.

More formally, find:

* The largest l1_penalty that has more non-zeros than ‘max_nonzeros’ (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)Store this value in the variable **‘l1_penalty_min’** (we will use it later)
* The smallest l1_penalty that has fewer non-zeros than ‘max_nonzeros’ (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)Store this value in the variable **‘l1_penalty_max’** (we will use it later)

Hint: there are many ways to do this, e.g.:

* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of l1_penalty and inspecting it to find the appropriate boundaries.

In [12]:
l1_penalty_max = min(df_max['l1'])
l1_penalty_min = max(df_min['l1'])

l1_penalty_min, l1_penalty_max

(127.42749857031335, 263.6650898730358)

<font color = 'steelblue'><b> Quiz :  What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively?

<font color = 'mediumvioletred'><b> Answer : The values are as below : </b></font>
<font color = 'slategray'><b>
- l1_penalty_min = 127.42749857031335
- l1_penalty_max = 263.6650898730358
    </b></font>

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [13]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

not_zeros = 0
non_zeros = []
best_RSS = []
best_l1 = []


for l1_penalty in l1_penalty_values :
    reg_range = Lasso(alpha=l1_penalty,normalize=True,max_iter=500000)
    model_range = reg_range.fit(train[all_features], train['price'])
    
    predictions = reg_range.predict(validation[all_features])
    
    residuals = predictions - validation['price']
    RSS = np.sum(np.square(residuals))
    
    if model_range.intercept_ != 0 : 
        not_zeros = np.count_nonzero(model_range.coef_) + 1
        print('L1 Penalty : ', l1_penalty, '\tRSS : ', RSS, '\nNumber of non-zero coefficients : ', 
              not_zeros, '\n')
    
    elif model.intercept_ == 0 :
        not_zeros = np.count_nonzero(model_range.coef_)
        print('L1 Penalty : ', l1_penalty, '\nNumber of non-zero coefficients : ', not_zeros, '\n')

    best_l1.append(l1_penalty)
    best_RSS.append(RSS)
    non_zeros.append(not_zeros)

L1 Penalty :  127.42749857031335 	RSS :  435374677102680.6 
Number of non-zero coefficients :  10 

L1 Penalty :  134.5978981125619 	RSS :  437009229124471.25 
Number of non-zero coefficients :  10 

L1 Penalty :  141.76829765481045 	RSS :  438236128386912.2 
Number of non-zero coefficients :  8 

L1 Penalty :  148.938697197059 	RSS :  439158937799660.1 
Number of non-zero coefficients :  8 

L1 Penalty :  156.10909673930755 	RSS :  440037365263316.56 
Number of non-zero coefficients :  7 

L1 Penalty :  163.2794962815561 	RSS :  440777489641605.1 
Number of non-zero coefficients :  7 

L1 Penalty :  170.44989582380464 	RSS :  441566698090139.94 
Number of non-zero coefficients :  7 

L1 Penalty :  177.6202953660532 	RSS :  442406413188666.3 
Number of non-zero coefficients :  7 

L1 Penalty :  184.79069490830176 	RSS :  443296716874314.9 
Number of non-zero coefficients :  7 

L1 Penalty :  191.96109445055032 	RSS :  444239780526141.6 
Number of non-zero coefficients :  7 

L1 Penalty

In [14]:
data = {'non_zeros' : non_zeros, 'L1 Penalty' : L1_penalties, 'RSS' :RSS_}
df = pd.DataFrame(data=data)
#print('Before Selecting :\n', df)
df = df[df['non_zeros'] == max_nonzeros]
#print('\nAfter Selecting :\n', df)


# The code below is used to answer the quiz question as below
lowest_rss = min(df['RSS'])
df = df[df['RSS']==lowest_rss]
#print('\nLowest DF:\n',df)
print('\nL1 Penalty', df['L1 Penalty'].item(), ' has is the lowest L1 Penalty with RSS of', df['RSS'].item())

data_coef = {'Features' : all_features, 'Coefficients' : model_range.coef_}
df_coef = pd.DataFrame(data=data_coef)
df_coef = df_coef[df_coef['Coefficients'] > 0]
print('Features with non-zero coefficients are : \n\n' ,df_coef)

#print('\n\nModel Coefficients :\n', pd.Series(model_range.coef_,index=all_features))


L1 Penalty 42.81332398719393  has is the lowest L1 Penalty with RSS of 406877258520204.6
Features with non-zero coefficients are : 

        Features   Coefficients
3   sqft_living     164.307714
9    waterfront  420588.370815
10         view   40094.951895
12        grade  104485.991764


<font color = 'steelblue'><b> Quiz 1:  What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?

<font color = 'mediumvioletred'><b> Answer 1 : The value for `l1_penalty` in range with sparsity equal to `max_zeros` is 42.81332398719393 </b></font>

<br/>

<font color = 'steelblue'><b> Quiz 2:  What features in this model have non-zero coefficients?

<font color = 'mediumvioletred'><b> Answer 2 : `sqft_living`, `waterfront`, `view` and `grade` are features with non-zero coefficients </b></font>